In [ ]:
import os
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import KFold
import cv2 as cv
import plotly.express as px
from tqdm.notebook import tqdm




import torch
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2


<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='color:#0D0D0D; background:#27BDEB; font-size: 40px;' role="tab" aria-controls="home"><br><center>1. Show data</center></h2>

In [ ]:
df = pd.read_csv('../input/paddy-disease-classification/train.csv')

df.sample(7)

In [ ]:
train_dir = '../input/paddy-disease-classification/train_images/'
df['path_jpeg'] = df.apply(lambda row: train_dir + row['label'] + '/' + row['image_id'], axis=1)
df.sample(7)

In [ ]:
plt.rcParams["font.family"] = 'serif'
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(18, 15),
                        subplot_kw={'xticks': [], 'yticks': []})

for ax in axes.flat:
    i = np.random.randint(df.shape[0])
    ax.imshow(plt.imread(df['path_jpeg'][i]))
    ax.set_title(df['label'][i],fontsize=16)

plt.show()

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(df['label'])
integer_mapping = {i: l for i, l in enumerate(le.classes_)}
df['label'] = le.transform(df['label'])

In [ ]:
integer_mapping

In [ ]:
df_gr = df.value_counts(['label'])
df_gr = df_gr.reset_index()
x = df_gr['label'].values
y = df_gr[0].values
plt.figure(figsize =(14, 7))
plt.bar(x, y)
plt.xticks(x, fontsize=14)
plt.yticks([y.min(), y.max(), y.mean()],fontsize=14)
plt.grid()
plt.show()


<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='color:#0D0D0D; background:#27BDEB; font-size: 40px;' role="tab" aria-controls="home"><br><center>2. K-Fold Stratified Data</center></h2>

In [ ]:
df['k_fold'] = np.nan

n_folds = 7
reversed = False

for label in df.label.unique():
    
    folds = list(range(n_folds))
    if reversed: folds.reverse()
    
    kf = KFold(n_splits=n_folds, random_state=42, shuffle=True)
    
    label_idxs = df[df.label==label].index
    
    kf.get_n_splits(label_idxs)

    for _, valid_index in kf.split(label_idxs):

        actual_fold = folds.pop(0)
        df_index = label_idxs[valid_index]
        df.loc[df_index, 'k_fold'] = actual_fold
    reversed = not reversed
        



In [ ]:
graphic = df.groupby(['label', 'k_fold']).size().reset_index()
graphic.columns = ['label', 'k_fold', 'count']
fig = px.bar(
    graphic, x="k_fold", y="count",
    color='label', barmode='group',
    height=400
)
fig.show()

<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='color:#0D0D0D; background:#27BDEB; font-size: 40px;' role="tab" aria-controls="home"><br><center>3. Create Augmentations</center></h2>

# How does it work? For example:



In [ ]:
# create transform  
transform = A.Compose([
    A.Rotate([-30,30]),
    A.RandomCrop(width=360, height=360),
    A.RandomBrightnessContrast(brightness_limit=[0.1,0.6], contrast_limit=[0.1,0.6], p=0.3),
    A.HorizontalFlip(p=0.5)
])
# this method return image-array
# transformed = transform(image=image)
# transformed.keys()

#create visualization
row, col = 3, 5
number = 1
plt.figure(figsize=(12, 8))
for r in range(row):
    # choose a random picture 
    i = np.random.randint(df.shape[0])
    image = cv.imread(df['path_jpeg'][i])
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    for c in range(col):
        # create transform
        if number % col == 1:
            plt.subplot(row, col, number)
            plt.imshow(image)
            plt.title('original')
        else:
            transformed_image = transform(image=image)['image']
            plt.subplot(row, col, number)
            plt.imshow(transformed_image)
            
        plt.xticks([]);
        plt.yticks([]);
        number += 1
                
plt.show()

In [ ]:
def transforms_method(image=None, image_size=360, train=True):
#     if image is None:
#         raise ValueError('image is NaN')
    if train:
        transforms = A.Compose([
            A.RandomCrop(height=image_size, width=image_size, always_apply=True),
            A.Rotate([-30,30], p=1),
            A.CoarseDropout(max_height=int(image_size * 0.17), max_width=int(image_size * 0.17),
                             min_holes=4, max_holes=9, p=0.7),
            A.RandomGridShuffle(grid=(2, 2), p=0.3),
            A.GaussianBlur(blur_limit=(3, 7), p=0.05),
            A.RandomSnow(p=0.05),
            A.RandomRain(p=0.05),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225],),
            ToTensorV2(),
        ])
    else:
        transforms = A.Compose([
        A.CenterCrop(height=image_size, width=image_size, always_apply=True),
        A.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225],),
        ToTensorV2(),
        ])
        
    return transforms

In [ ]:

i = np.random.randint(df.shape[0])

image = cv.imread(df['path_jpeg'][i])
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

transform = transforms_method(image)

transformed_image = transform(image=image)['image']
#PyTorch Tensor
plt.imshow(transformed_image.permute(1, 2, 0));

<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='color:#0D0D0D; background:#27BDEB; font-size: 40px;' role="tab" aria-controls="home"><br><center>4. Create Dataset</center></h2>

In [ ]:
class PaddyDataset(Dataset):
    def __init__(self, images_filepaths, targets, transform=None):
        self.images_filepaths = images_filepaths
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv.imread(image_filepath)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

        if self.transform is not None:
            image = self.transform(image=image)['image']
        
        label = torch.tensor(self.targets[idx]).long()
        return image, label

<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='color:#0D0D0D; background:#27BDEB; font-size: 40px;' role="tab" aria-controls="home"><br><center>5. Let's create a model</center></h2>

In [ ]:

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Thera are  %d GPU(s) available.' % torch.cuda.device_count())
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


In [ ]:
from torchvision import models
model = models.resnet50(pretrained=True)

# #freezy model_parameters 
# for param in model.parameters():
#     param.requires_grad = False
    
model.fc = nn.Sequential(
    nn.Dropout(0.1),
    nn.Linear(model.fc.in_features,  10)
    
)
# model

In [ ]:
model = model.to(device)
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),  lr = 10 **(-3))
epochs = 5
batch_size = 37

<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='color:#0D0D0D; background:#27BDEB; font-size: 40px;' role="tab" aria-controls="home"><br><center>6. Training model</center></h2>

In [ ]:
list_of_loss_train = []
list_of_loss_val = []
list_of_acc = []
for epoch_i in range(1, epochs + 1):
    
    
    print(f'---------------------epoch:{epoch_i}/{epochs}---------------------')
    for valid_fold in tqdm(range(n_folds)):
        
            
        train = df[df['k_fold'] != valid_fold]
        valid = df[df['k_fold'] == valid_fold]

        X_train = train['path_jpeg']
        y_train = train['label']
        X_valid = valid['path_jpeg']
        y_valid = valid['label']

        train_dataset = PaddyDataset(
            images_filepaths=X_train.values,
            targets=y_train.values,
            transform=transforms_method()
        )

        valid_dataset = PaddyDataset(
            images_filepaths=X_valid.values,
            targets=y_valid.values,
            transform=transforms_method(train=False)
        )
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
        
        total_train_loss = 0
        total_eval_loss = 0
        summa = 0
        
        model.train()
        
        for batch in (train_loader):
            X_batch = batch[0].to(device)
            y_batch = batch[1].to(device)

            optimizer.zero_grad()
            res = model.forward(X_batch)
            loss_value = loss(res, y_batch.long())
            loss_value.backward()

            total_train_loss += loss_value
            optimizer.step()
            
        model.eval()
        for batch in (valid_loader):
            X_batch = batch[0].to(device)
            y_batch = batch[1].to(device)

            with torch.no_grad(): 
                res = model.forward(X_batch)
                preds = torch.max(F.softmax(res, dim=1), dim=1)
                correct= torch.eq(preds[1], y_batch)
                summa += torch.sum(correct).item()
                loss_value = loss(res, y_batch.long())
                total_eval_loss += loss_value
                
        avg_train_loss = total_train_loss / len(train_dataset)
        avg_val_loss = total_eval_loss / len(valid_dataset)
        acc = summa / len(valid_dataset)
        
        list_of_loss_train.append(avg_train_loss.cpu().detach().numpy())
        list_of_loss_val.append(avg_val_loss.cpu().detach().numpy())
        list_of_acc.append(acc)
        
        print(f'epoch: {epoch_i}, path: {valid_fold+1}/{n_folds} acc:{acc :.2%} ({summa}/{len(valid_dataset)}), loss_train:{avg_train_loss:.5f}, loss_valid:{avg_val_loss:.5f}')

            

<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='color:#0D0D0D; background:#27BDEB; font-size: 40px;' role="tab" aria-controls="home"><br><center>7. Visualization of results</center></h2>

In [ ]:
acc_max = max(list_of_acc)
loss_train_min = min(list_of_loss_train)
loss_val_min = min(list_of_loss_val)

In [ ]:
plt.figure(figsize=(20, 5))

plt.subplot(1,2,1)
plt.title('Loss_score', fontsize = 15 );
plt.grid(True)
plt.xlabel('epoch', fontsize=14)
plt.plot(list_of_loss_train, color='red', label = f'min_value:{loss_train_min:.5f}');
plt.plot(list_of_loss_val, color='blue', label = f'min_value:{loss_val_min:.5f}');
plt.xticks(range(epochs * n_folds ));
plt.legend();

plt.subplot(1,2,2)
plt.title('Acc_score', fontsize = 15 );
plt.grid(True)
plt.xlabel('epoch', fontsize=14);
plt.plot(list_of_acc , color='darkblue', label = f'max_acc:{acc_max:.2%}');
plt.xticks(range(epochs * n_folds ));
plt.legend();

plt.show()

<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='color:#0D0D0D; background:#27BDEB; font-size: 40px;' role="tab" aria-controls="home"><br><center>8. Submit predictions</center></h2>

In [ ]:
submission_dir = '../input/paddy-disease-classification/test_images/'

In [ ]:
%%time
model.eval()
image_ids, labels = [], []
for (dirpath, dirname, filenames) in os.walk(submission_dir):
    filenames.sort()
    for imade_id in tqdm(filenames):
        image_filepath = dirpath + imade_id
        
        image = cv.imread(image_filepath)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        
        transform = transforms_method(image, train=False)
        transformed_image = transform(image=image)['image']
#       plt.imshow(transformed_image.permute(1, 2, 0));

        res = model.forward(transformed_image.unsqueeze(0).to(device))
        pred = torch.max(F.softmax(res, dim=1), dim=1)[1].to('cpu')
        pred_label = integer_mapping[pred.numpy()[0]]

        image_ids.append(imade_id)
        labels.append(pred_label)

In [ ]:
submission = pd.DataFrame({
    'image_id': image_ids,
    'label': labels,
})

In [ ]:
submission.to_csv("submission.csv", index=False, header=True)

In [ ]:
submission.head()